In [ ]:
using Plots; gr()
using FUSE
using OrderedCollections
include("hf_mapper.jl")

In [ ]:
# FUSE.logging(Logging.Info; actors=Logging.Info);
dd = IMAS.dd();
# ini, act = FUSE.case_parameters(:ITER, init_from = :ods) # ok
# ini, act = FUSE.case_parameters(:ITER, init_from = :scalars) # ok
# ini, act = FUSE.case_parameters(:FPP) # ok
# ini, act = FUSE.case_parameters(:DTT) # ok
ini, act = FUSE.case_parameters(:D3D) # ok
# ini, act = FUSE.case_parameters(:KDEMO) # ok
# ini, act = FUSE.case_parameters(:ARC) # ok 
# ini, act = FUSE.case_parameters(:SPARC) # ok
# ini, act = FUSE.case_parameters(:STEP) # ok
dd = FUSE.init(ini,act);
# FUSE.digest(dd);
1;

In [ ]:
RA = dd.equilibrium.time_slice[].global_quantities.magnetic_axis.r
ZA = dd.equilibrium.time_slice[].global_quantities.magnetic_axis.z
# wall_r = IMAS.first_wall(dd.wall).r
# wall_z = IMAS.first_wall(dd.wall).z
# dd.wall.description_2d[1].limiter.unit[1].outline.r = [maximum(wall_r), maximum(wall_r), minimum(wall_r), minimum(wall_r), maximum(wall_r), maximum(wall_r)]
# dd.wall.description_2d[1].limiter.unit[1].outline.z = [ZA, minimum(wall_z),minimum(wall_z),maximum(wall_z),maximum(wall_z),ZA]
# # FUSE.wall_from_eq!(dd.wall, dd.equilibrium.time_slice[], 0.1; upper_divertor = false, lower_divertor = true )
# display(plot(dd.wall))
wall_r = IMAS.first_wall(dd.wall).r
wall_z = IMAS.first_wall(dd.wall).z

1


In [ ]:
# define q(r), but could be arbitrary. Here double exponential
R0 = dd.equilibrium.time_slice[].boundary.geometric_axis.r # R magentic axis 
a = dd.equilibrium.time_slice[].boundary.minor_radius      # minor radius
l = 0.003 # decay length of first exponential
l2 = 0.1  # decay length of second exponential
frac = 0.2 # fraction of power flowing in the second esponential
N = 2 # imbalance factor (in/out)  1 < N < 2
Bt_omp = dd.equilibrium.time_slice[].global_quantities.vacuum_toroidal_field.b0*R0/(R0+a)

r = collect(LinRange(0,6*maximum([l,l2]),10000)) # Ideally: from 0 to r max grid - r_separatrix_OMP
# double exponential
q = IMAS.power_sol(dd)     /2/N/π/(R0+a)/l /sin(atan(IMAS.Bpol_omp(dd.equilibrium.time_slice[])/abs(Bt_omp)))*exp.(-r./l)+
    IMAS.power_sol(dd)*frac/2/N/π/(R0+a)/l2/sin(atan(IMAS.Bpol_omp(dd.equilibrium.time_slice[])/abs(Bt_omp)))*exp.(-r./l2);

plot(r,log10.(q))
plot!(ylabel = "log₁₀(q)", xlabel = "r - rₛₑₚ")
plot!(legend = :none)

In [ ]:
# stagger wall
# dd.wall.description_2d[1].limiter.unit[1].outline.z = dd.wall.description_2d[1].limiter.unit[1].outline.z .+ 0.3
# dd.wall.description_2d[1].limiter.unit[1].outline.z = dd.wall.description_2d[1].limiter.unit[1].outline.z .- 0.4
1


In [ ]:
levels = 20
psi_levels, R, p_levels  =  IMAS.find_levels_from_P(dd,r,q,levels) # find psi_levels to discretize properly q(r)
@show p_levels
add_psi =  IMAS.find_levels_from_wall(dd) # add psi of the points in the wall
# @show add_psi
psi_levels = unique!(sort!(vcat(psi_levels, add_psi)))
# display(psi_levels)

SOL = IMAS.sol(dd, levels = psi_levels); # Build SOL with the chosen psi

plot(dd.wall, lw=1)
plot!(SOL)
# plot!(SOL[:lfs_far][end])
plot!(legend = :none)
plot!(xlim = (minimum(wall_r)*0.9, maximum(wall_r)*1.1))
# plot!(ylim = (-5,5))
# plot!(ylim = (0.5,1), xlim = (8,8.5))
# surfaces, _ = IMAS.flux_surface(dd.equilibrium.time_slice[],14.1147162341807925, :open)
# plot!(dd.equilibrium.time_slice[].profiles_2d)
# for surface in surfaces
# plot!(surface, color = :black)
# end
# surfaces, _ = IMAS.flux_surface(dd.equilibrium.time_slice[],2.9491989681605846, :open)
# for surface in surfaces
# plot!(surface, color = :black)
# end

# length(surfaces)




In [ ]:

_, _, PSI_interpolant = IMAS.ψ_interpolant(dd.equilibrium.time_slice[].profiles_2d)
psi_low, psi_up = IMAS.find_psi_tangent_omp(dd.equilibrium.time_slice[],wall_r,wall_z,PSI_interpolant)
_, psi_sep = IMAS.find_psi_boundary(dd.equilibrium.time_slice[])
psi_2nd  =  IMAS.find_psi_2nd_separatrix(dd)
psi_last_diverted = IMAS.find_psi_last_diverted(dd)


plot(dd.wall, lw=1)
# plot!([8.142406627710745, 6.38759043904455, 6.578835273420422, 6.770080107796293, 6.9613249421721655, 7.116071319012841, 7.2606703401688275, 7.405269361324812, 7.5498683824808, 7.694467403636786, 7.839066424792772, 7.983665445948757, 8.128264467104746, 8.142406627710745, 8.272175866772006, 8.336620923584087, 8.354618054249304, 8.372615184914519, 8.390612315579736, 8.392909207612265, 8.392909207612265, 8.392909207612263, 8.392909207612263, 8.392909207612265, 8.392909207612265, 8.392909207612263, 8.3186157547294, 8.234999198891499, 8.1513826430536, 8.0677660872157, 7.984149531377801, 7.9005329755399, 7.816916419702, 7.67195658445048, 7.494785035246251, 7.3176134860420206, 7.1404419368377905, 6.96327038763356, 6.786098838429332, 8.392909207612265],
# [-10.0, -3.007970116041419, -2.859641465933984, -2.711312815826552, -2.5629841657191172, -2.3776793089496606, -2.1835991755024704, -1.9895190420552828, -1.7954389086080924, -1.601358775160905, -1.4072786417137146, -1.2131985082665273, -1.019118374819337, -1.0, -0.8245687472011171, -0.5951433746800361, -0.3537887012097112, -0.11243402773938653, 0.12892064573093814, 0.3708598688073396, 0.6128846095223088, 0.8549093502372795, 0.8549093502372795, 1.0, 1.0969340909522487, 1.3389588316672176, 1.5677421562860026, 1.7948638124704255, 2.0219854686548486, 2.249107124839272, 2.4762287810236954, 2.7033504372081176, 2.930472093392541, 3.12066975138505, 3.2855521860793533, 3.450434620773657, 3.61531705546796, 3.7801994901622638, 3.9450819248565674, 10.0],
# color = :green)
plot!(legend = :none)
# plot!(ylim = (2,3.5), xlim = (7,8))

surfaces, _ = IMAS.flux_surface(dd.equilibrium.time_slice[],psi_low, :open)
for surface in surfaces
plot!(surface, color = :red)
end
surfaces, _ = IMAS.flux_surface(dd.equilibrium.time_slice[],psi_sep, :open)
for surface in surfaces
plot!(surface, color = :black)
end
# surfaces, _ = IMAS.flux_surface(dd.equilibrium.time_slice[],psi_2nd, :open)
# for surface in surfaces
# plot!(surface, color = :blue)
# end
# surfaces, _ = IMAS.flux_surface(dd.equilibrium.time_slice[],psi_last_diverted[1], :open)
# for surface in surfaces
# plot!(surface, color = :darkorange)
# end
# surfaces, _ = IMAS.flux_surface(dd.equilibrium.time_slice[],11.834792365178698, :open)
# for surface in surfaces
# plot!(surface, color = :red)
# end
plot!([0, maximum(wall_r)*1.1], [1, 1]*ZA, ls = :dash, color = :grey)
plot!(xlim = (minimum(wall_r)*0.9, maximum(wall_r)*1.1))
# plot!(ylim = (-5,5))
display(plot!())


In [ ]:
eqt = dd.equilibrium.time_slice[]
wall_r = IMAS.first_wall(dd.wall).r
wall_z = IMAS.first_wall(dd.wall).z;

# HF = WallHFMapper(eqt,SOL,wall_r,wall_z,r,q); # map heat flux
1

In [ ]:
### different recepies for plotting
# plot(HF)
# plot(HF, q = :parallel)
# plot(HF, q = :both)
# plot(HF, q = :both, cat = true)
# plot(HF, q = :both, plot_type = :scatter, cat = true)
# display(plot(HF, which_plot = :oneD))
# plot(HF, which_plot = :oneD, q = :wall)
# plot(HF, which_plot = :oneD, q = :both)

In [ ]:
# l = 0.005
# l2 = 0.05
# frac = 0.2
# r = collect(LinRange(0,7*maximum([l,l2]),10000)) # Ideally: from r separatrix to r max grid
# q = IMAS.power_sol(dd)     /2/N/π/(R0+a)/l /sin(atan(IMAS.Bpol_omp(eqt)/abs(Bt_omp)))*exp.(-r./l)+
#     IMAS.power_sol(dd)*frac/2/N/π/(R0+a)/l2/sin(atan(IMAS.Bpol_omp(eqt)/abs(Bt_omp)))*exp.(-r./l2)
HF = WallHFMapper(dd,r,q, merge_wall = true, N = Int(1E6));

# plot(plot(HF, which_plot = :oneD), plot(HF, plot_type = :scatter))


# plot!(xlim = (20,22))
1

# @show maximum(HF.q_core_rad) 
# @show maximum(HF.q_core_rad./4.9784699141794324e8) 
# display(plot(HF, which_plot = :twoD, q = :core_rad, plot_type = :scatter))

# plot(size = (4000,500))
plot(log10.(HF.q_core_rad .+1))
plot!(log10.(HF.q_core_rad .+1), seriestype = :scatter, ms =1)
plot!(legend = :none)


In [ ]:
sum(HF.q_core_rad .<0)

In [ ]:

_, psi_separatrix = IMAS.find_psi_boundary(dd)
surface, _ = IMAS.flux_surface(eqt,psi_separatrix, :open)
p1 = plot(HF, which_plot = :oneD, q = :core_rad)

p2 = plot(HF, q = :corerad)

ll = @layout [a{0.6w,0.9h} b{0.4w}]
p = plot(; layout=ll, size=(1000, 500))
plot!(HF, which_plot = :oneD, q = :core_rad; subplot =1)

plot!(HF, q = :core_rad, plot_type =  :scatter; subplot =2)
display(plot!(surface[1], color = :grey; subplot =2))
1

p1 = plot(HF, which_plot = :oneD, q = :part)

p2 = plot(HF, q = :part)

ll = @layout [a{0.6w,0.9h} b{0.4w}]
p = plot(; layout=ll, size=(1000, 500))
plot!(HF, which_plot = :oneD, q = :part; subplot =1)
# plot!(xlim=(16,20); subplot = 1)
plot!(HF, q = :part, plot_type =  :path; subplot =2)
# plot!(ylim = (1.0,Inf), xlim=(0.9,1.4); subplot = 2)
plot!([0, maximum(HF.r)*1.1], [ZA,ZA], ls = :dash, color = :grey; subplot = 2)
# plot!([0, maximum(HF.r)*1.1], -2*[ZA,ZA], ls = :dash, color = :grey; subplot = 2)
# plot!(dd.wall, lw =1; subplot = 2)
display(plot!(surface[1], color = :grey; subplot =2))
# plot( size=(4000,500))
# display(plot!(HF.z[2:end]-HF.z[1:end-1], st = :scatter, ms =1, label = ""))
# plot( size=(4000,500))
# plot!(HF.z)

In [ ]:
wall_r[1:10]



In [ ]:
HF.q_part[HF.q_part.<0]



In [ ]:
@show maximum(HF.q_core_rad/1e6) 
@show sum(HF.q_core_rad)/length(HF.q_core_rad)
@show maximum(HF.q_core_rad/1e6) * 8e7/4.133741864504699e7
@show sum(HF.q_core_rad)/length(HF.q_core_rad) * 8e7/4.133741864504699e7
@show 0.048837751067294446 * 8e7/4.133741864504699e7
# display(plot(HF, which_plot = :twoD, q = :core_rad, plot_type = :scatter))

plot(size = (4000,500))
plot!(HF.s, (HF.q_core_rad)./4.9784699141794324e8)
plot!(HF.s,(HF.q_core_rad)./4.9784699141794324e8, seriestype = :scatter, ms = 1)
plot!(legend = :none)
# plot!(HF, which_plot = :oneD, q = :core_rad)

In [ ]:


N = Int(1E6)
act.ActorNeutronics.N = N
act.ActorNeutronics.do_plot = true
FUSE.ActorNeutronics(dd,act)
# @show length(dd.neutronics.first_wall.r)
xlim = extrema(dd.neutronics.first_wall.r)
xlim = (xlim[1] - ((xlim[2] - xlim[1]) / 10.0), xlim[2] + ((xlim[2] - xlim[1]) / 10.0))
l = @layout [a{0.3w} b{0.6w,0.9h}]
p = plot(; layout=l, size=(900, 400))
plot!(dd.neutronics.time_slice[].wall_loading; cx=false, subplot=2, ylabel="")


In [ ]:
# ###### Works with all cases available
# dd = IMAS.dd();
# # ini, act = FUSE.case_parameters(:ITER, init_from = :ods) 
# # ini, act = FUSE.case_parameters(:ITER, init_from = :scalars) 
# # ini, act = FUSE.case_parameters(:FPP, version = :v2) 
# # ini, act = FUSE.case_parameters(:DTT) # ok
# # ini, act = FUSE.case_parameters(:KDEMO) # ok
# # ini, act = FUSE.case_parameters(:ARC) # ok 
# # ini, act = FUSE.case_parameters(:SPARC) # ok
# # ini, act = FUSE.case_parameters(:STEP) # ok
# ini, act = FUSE.case_parameters(:D3D) 
# dd = FUSE.init(ini,act);
# 1;

In [ ]:
# l = 0.003
# l2 = 0.1
# frac = 0.15
# r = collect(LinRange(0,6*maximum([l,l2]),10000)) # Ideally: from r separatrix to r max grid
# q = IMAS.power_sol(dd)     /2/N/π/(R0+a)/l /sin(atan(IMAS.Bpol_omp(eqt)/abs(Bt_omp)))*exp.(-r./l)+
#     IMAS.power_sol(dd)*frac/2/N/π/(R0+a)/l2/sin(atan(IMAS.Bpol_omp(eqt)/abs(Bt_omp)))*exp.(-r./l2)

# HF = WallHFMapper(dd,r,q, merge_wall = true);
# eqt = dd.equilibrium.time_slice[]
# _, psi_separatrix = IMAS.find_psi_boundary(dd)
# psi_2ndseparatrix = IMAS.find_psi_2nd_separatrix(dd)
# surface, _  = IMAS.flux_surface(eqt,psi_2ndseparatrix, :open)
# surface2, _ = IMAS.flux_surface(eqt,psi_separatrix,:open);
# p1 = plot(dd.wall,lw=1)
# plot!(collect(surface), color = :gray)
# plot!(collect(surface2), color = :gray)
# plot!(HF, plot_type = :path)
# plot(plot(HF, which_plot = :oneD), p1)

# # plot(HF, which_plot = :oneD)


In [ ]:
@show (HF.z[1], HF.z[end])

In [ ]:
plot(HF.r,HF.z,st =:scatter, aspect_ratio = :equal, ms = 1)
plot!(HF.r,HF.z)
plot!(legend = :none)

In [ ]:

_, psi_separatrix = IMAS.find_psi_boundary(dd)
surface, _ = IMAS.flux_surface(eqt,psi_separatrix, :open)
# plot(dd.wall, lw =1)
# plot!(SOL[:lfs_far][end], colorbar = false)
# plot(HF, q = :part, which_plot = :oneD)
plot(HF, q = :part, plot_type = :scatter)
plot!(surface[1], color = :gray)
